In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
# Loading CSV file
df_amazon = pd.read_csv ("data.csv", sep=",")
df_amazon=df_amazon[['Summary','Classification']]
df_amazon=df_amazon.dropna()
df_amazon.shape


(1280, 2)

In [ ]:
for i in range(len(df_amazon['Summary'])):
  if type(df_amazon['Summary'][4])==None:
    print("im here")

In [ ]:
!pip install spacy && python -m spacy download en 


     |████████████████████████████████| 12.0 MB 4.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import string,spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [ ]:
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    if text==None:
      return ""
    # Removing spaces and converting text into lowercase
    return str(text).lower()

In [ ]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))


In [ ]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)


In [ ]:
from sklearn.model_selection import train_test_split

X = df_amazon['Summary'] # the features we want to analyze
ylabels = df_amazon['Classification'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [ ]:
print(X_train)

360    Upper case conversion of special characters ca...
317    [app.class.php] Patch to add Marketplace Datab...
263    trace-malloc broken on fxdbug-win32-tbox (due ...
333    review: Move the quip list into the database; ...
289    Shared library suffix incorrectly assumed to b...
                             ...                        
568                    Extending an abstract class fails
548             DBCS: shown up GnomeVFS-WARNING messages
797    DE:PR1 investorworld spacing after "Anmelden" ...
196    Investigate Dependency graph builder as resour...
518    Generate some of Roadmap from project informat...
Name: Summary, Length: 896, dtype: object


In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7f86ce759ed0>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f86dde70ef0>)),
                ('classifier', LogisticRegression())])

In [ ]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
# print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
# print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.515625


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

classifier2 = SVC(kernel='linear')

# Create pipeline using Bag of Words
pipe1 = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier2)])

# model generation
pipe1.fit(X_train,y_train)


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7f86ddceb850>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f86dde70ef0>)),
                ('classifier', SVC(kernel='linear'))])

In [ ]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe1.predict(X_test)

# Model Accuracy
print("SVM Accuracy:",metrics.accuracy_score(y_test, predicted))
# print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
# print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

SVM Accuracy: 0.5260416666666666


In [ ]:
from sklearn.ensemble import RandomForestClassifier as rf

classifier3 = rf()

# Create pipeline using Bag of Words
pipe2 = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier3)])

# model generation
pipe2.fit(X_train,y_train)
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe2.predict(X_test)

# Model Accuracy
print("Random forest Accuracy:",metrics.accuracy_score(y_test, predicted))
# print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
# print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Random forest Accuracy: 0.53125


In [ ]:
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.naive_bayes import MultinomialNB
classifier3 = MultinomialNB()

# Create pipeline using Bag of Words
pipe2 = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier3)])

# model generation
pipe2.fit(X_train,y_train)
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe2.predict(X_test)

# Model Accuracy
print("Naive Bayes Accuracy:",metrics.accuracy_score(y_test, predicted))
# print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
# print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Naive Bayes Accuracy: 0.5260416666666666


In [ ]:
print(X_test)

795     JA: The orange gif in center of side bar panel...
1083                           read of unitialized memory
887     PHP Preferences - in servers page - add a "mak...
1190               Security access to server and database
762                         crash on startup with IBM JRE
                              ...                        
255     browser doesn't handle simple response of HTTP...
6                           No support for Apache 2.2 yet
487               Can't compile with bundled apr/apr-util
291      JA:PR2 Bargain America link incorrect in sidebar
847                        border width in tables adds up
Name: Summary, Length: 384, dtype: object


In [ ]:
import numpy as np
import re
import nltk
nltk.download('wordnet')
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,ylabels , test_size=0.2, random_state=0)
classifier.fit(X_train, y_train) 

LogisticRegression()

In [ ]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 4  1  0 14  0  0  0  0  0  0  0  0  0  1  0]
 [ 2  4  0 26  0  0  0  0  0  0  0  0  0  1  0]
 [ 1  0  2  3  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  6  0 85  0  6  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0 23  0 18  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  2  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  6  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  1  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  3  0  1  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0  7  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  2  0  0  0  0  0  0  0 10  0]
 [ 0  1  0  1  0  0  0  0  0  0  0  0  0  0  1]]
                              precision    recall  f1-score   support

                   Add issue       0.33      0.20      0.25        20
         Configuration issue       0.27      0.12      0.17        33
      Database-related issue       1.00      0.33      0.50         

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
